In [1]:
from nltk import word_tokenize, pos_tag, ne_chunk

c:\users\subhojeet.rudra\appdata\local\continuum\miniconda3\envs\myenv\lib\site-packages\sklearn\utils\fixes.py:313: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  _nan_object_mask = _nan_object_array != _nan_object_array


In [3]:
sentence="Incident IR53463667 is created because job MMP_GHG_040 failed or abended due to server overload"

In [10]:
print(ne_chunk(pos_tag(word_tokenize(sentence))))

(S
  (PERSON Incident/NNP)
  (ORGANIZATION IR53463667/NNP)
  is/VBZ
  created/VBN
  because/IN
  job/NN
  (ORGANIZATION MMP_GHG_040/NNP)
  failed/VBD
  or/CC
  abended/VBD
  due/JJ
  to/TO
  server/VB
  overload/NN)


In [12]:
import nltk
doc = sentence
# tokenize doc
tokenized_doc = nltk.word_tokenize(doc)
 
# tag sentences and use nltk's Named Entity Chunker
tagged_sentences = nltk.pos_tag(tokenized_doc)
ne_chunked_sents = nltk.ne_chunk(tagged_sentences)
 
# extract all named entities
named_entities = []
for tagged_tree in ne_chunked_sents:
    if hasattr(tagged_tree, 'label'):
        entity_name = ' '.join(c[0] for c in tagged_tree.leaves()) #
        entity_type = tagged_tree.label() # get NE category
        named_entities.append((entity_name, entity_type))
print(named_entities)

[('Incident', 'PERSON'), ('IR53463667', 'ORGANIZATION'), ('MMP_GHG_040', 'ORGANIZATION')]


In [51]:
data = [(['Linux', 'is', 'the', 'best', 'OS'], ['OS','IR','IR','IR','IR']),
        (['Among', 'windows','and ', 'mac',  'ubuntu','is','better'], ['IR','OS','IR','OS','OS','IR','IR']),
        #(['I','am','planning','to','buy','mac'],['IR','IR','IR','IR','IR','OS']),
(['Ubuntu', 'is', 'my', 'favourite', 'OS'], ['OS','IR','IR','IR','IR'])]
corpus = []
for (doc, tags) in data:
    doc_tag = []
    for word, tag in zip(doc,tags):
        doc_tag.append((word, tag))
    corpus.append(doc_tag)
print(corpus)

[[('Linux', 'OS'), ('is', 'IR'), ('the', 'IR'), ('best', 'IR'), ('OS', 'IR')], [('Among', 'IR'), ('windows', 'OS'), ('and ', 'IR'), ('mac', 'OS'), ('ubuntu', 'OS'), ('is', 'IR'), ('better', 'IR')], [('Ubuntu', 'OS'), ('is', 'IR'), ('my', 'IR'), ('favourite', 'IR'), ('OS', 'IR')]]


In [52]:
def doc2features(doc, i):
    word = doc[i][0]
    
    # Features from current word
    features={
        'word.word': word,
    }
    # Features from previous word
    if i > 0:
        prevword = doc[i-1][0]
        features['word.prevword'] = prevword
    else:
        features['BOS'] = True # Special "Beginning of Sequence" tag
        
    # Features from next word
    if i < len(doc)-1:
        nextword = doc[i+1][0]
        features['word.nextword'] = nextword
    else:
        features['EOS'] = True # Special "End of Sequence" tag
    return features
 
def extract_features(doc):
    return [doc2features(doc, i) for i in range(len(doc))]
 
X = [extract_features(doc) for doc in corpus]
print(X)
 

[[{'word.word': 'Linux', 'word.nextword': 'is', 'BOS': True}, {'word.word': 'is', 'word.nextword': 'the', 'word.prevword': 'Linux'}, {'word.word': 'the', 'word.nextword': 'best', 'word.prevword': 'is'}, {'word.word': 'best', 'word.nextword': 'OS', 'word.prevword': 'the'}, {'word.word': 'OS', 'EOS': True, 'word.prevword': 'best'}], [{'word.word': 'Among', 'word.nextword': 'windows', 'BOS': True}, {'word.word': 'windows', 'word.nextword': 'and ', 'word.prevword': 'Among'}, {'word.word': 'and ', 'word.nextword': 'mac', 'word.prevword': 'windows'}, {'word.word': 'mac', 'word.nextword': 'ubuntu', 'word.prevword': 'and '}, {'word.word': 'ubuntu', 'word.nextword': 'is', 'word.prevword': 'mac'}, {'word.word': 'is', 'word.nextword': 'better', 'word.prevword': 'ubuntu'}, {'word.word': 'better', 'EOS': True, 'word.prevword': 'is'}], [{'word.word': 'Ubuntu', 'word.nextword': 'is', 'BOS': True}, {'word.word': 'is', 'word.nextword': 'my', 'word.prevword': 'Ubuntu'}, {'word.word': 'my', 'word.nextwor

In [53]:
def get_labels(doc):
    return [tag for (token,tag) in doc]
y = [get_labels(doc) for doc in corpus]
print(y)

[['OS', 'IR', 'IR', 'IR', 'IR'], ['IR', 'OS', 'IR', 'OS', 'OS', 'IR', 'IR'], ['OS', 'IR', 'IR', 'IR', 'IR']]


In [57]:
import sklearn_crfsuite
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=20,
    all_possible_transitions=False,
)
crf.fit(X, y);

In [58]:
#test = [['CentOS', 'is', 'my', 'favourite', 'OS']]
test = [['I','am','planning','to','buy','centos']]

X_test = extract_features(test)
X_test[0]['word.word']='centos'
print(dict(X_test[0]))
print(crf.predict_single(X_test))

{'word.word': 'centos', 'EOS': True, 'BOS': True}
['IR']
